In [1]:
import yoda
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams['text.usetex'] = True
import numba as nb

Welcome to JupyROOT 6.30/02


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def return_data_hist_yoda(hist_name='d01-x01-y01'):
    # print('data hist name = ', hist_name)
    data_hist_path = os.path.join(os.getcwd(), 'rivet_histograms', 'data', 'ALEPH_1996_S3486095.yoda')
    tree_name_data='/REF/ALEPH_1996_S3486095'
    yoda_file = yoda.read(data_hist_path)
    full_hist_name = '%s/%s' % (tree_name_data, hist_name)
    yoda_hist = yoda_file[full_hist_name]
    hist_edges = yoda_hist.xVals()
    hist_values = yoda_hist.yVals()
    hist_errs = yoda_hist.yErrs()[:,0]
    return hist_edges, hist_values, hist_errs

In [3]:
def return_simulation_hist_yoda(filename = 'ALEPH_1996_S3486095_card_0.yoda', hist_name='d01-x01-y01'):
    # print('simulation hist name = ', hist_name)
    simulation_hist_path = os.path.join(os.getcwd(), 'rivet_histograms', 'simulation', filename)
    yoda_file = yoda.read(simulation_hist_path)
    tree_name_simulation='/ALEPH_1996_S3486095'
    
    full_hist_name = '%s/%s' % (tree_name_simulation, hist_name)
    yoda_hist = yoda_file[full_hist_name]
    hist_edges = yoda_hist.xVals()
    hist_values = yoda_hist.yVals()
    hist_errs = yoda_hist.yErrs()
    return hist_edges, hist_values, hist_errs

In [4]:
def get_hist_names():
    hist_names=[]
    for i in range(1,43):
        hist_names.append('d{:02d}-x01-y01'.format(i) )
        #13-16 and 24 not present in yoda files
    hist_names.remove('d13-x01-y01')
    hist_names.remove('d14-x01-y01')
    hist_names.remove('d15-x01-y01')
    hist_names.remove('d16-x01-y01')
    hist_names.remove('d24-x01-y01')
    # d25-x01-y01 gives error of unequal lengths
    # hist_names.remove('d25-x01-y01')
    # hist_names.remove('d26-x01-y01')
    # hist_names.remove('d27-x01-y01')
    #19-23 are empty
    hist_names.remove('d19-x01-y01')
    hist_names.remove('d20-x01-y01')
    hist_names.remove('d21-x01-y01')
    hist_names.remove('d22-x01-y01')
    hist_names.remove('d23-x01-y01')
    #36 gives a huge chi2 (=10^18) for unknown reason
    hist_names.remove('d36-x01-y01')
    # 41
    hist_names.remove('d41-x01-y01')
    hist_names.remove('d42-x01-y01')

    return hist_names

In [5]:
@nb.njit
def chi2(data_hist, sim_hist, sim_errs, print_=True):
    assert len(data_hist) == len(sim_hist)
    select = sim_hist > 0.0
    hsim = sim_hist[select]
    hdat = data_hist[select]
    sim_errs = sim_errs[select]
    
    num = (hdat - hsim)**2
    den = sim_errs**2 
    
    chi_2 =(num/den).sum()
    chi_2_per_dof =chi_2/len(hdat) 


    chi2 = np.sqrt(chi_2_per_dof)

    
    if print_==True:
        print('scaled chi2 per dof = \t', chi2)
    return chi2


In [7]:
def overall_chi2_observed_pred(sim_yoda_file='ALEPH_1996_S3486095_card_0.yoda', print_=True):
    """ returns the chi2 between the actually observed data and the predicted simulation"""

    hist_names = get_hist_names()
    overall_chi2 = 0
    for hist_name in hist_names:
        data_hist_edges, data_hist, data_hist_errs = return_data_hist_yoda(hist_name=hist_name)
        simulation_hist_edges, simulation_hist, simulation_hist_errs = return_simulation_hist_yoda(filename =sim_yoda_file , hist_name=hist_name)
        chi2_ = chi2(data_hist,simulation_hist, simulation_hist_errs, print_=print_)
        overall_chi2 = overall_chi2 +chi2_

    if print_==True:
        print('overall chi2= ', overall_chi2)
    return overall_chi2

In [8]:
def overall_chi2_shuffledpred_pred(sim_yoda_file='ALEPH_1996_S3486095_card_0.yoda', shuffled_sim_yoda_file = 'ALEPH_1996_S3486095_card_1.yoda',print_=True):
    """ returns the chi2 between the simulated pre"""

    hist_names = get_hist_names()
    overall_chi2 = 0
    for hist_name in hist_names:
        data_hist_edges, data_hist, data_hist_errs = return_simulation_hist_yoda(filename =shuffled_sim_yoda_file , hist_name=hist_name)
        simulation_hist_edges, simulation_hist, simulation_hist_errs = return_simulation_hist_yoda(filename =sim_yoda_file , hist_name=hist_name)
        
        chi2_ = chi2(data_hist,simulation_hist, simulation_hist_errs, print_=print_)
        overall_chi2 = overall_chi2 +chi2_

    if print_==True:
        print('overall chi2= ', overall_chi2)
    return overall_chi2

In [10]:
def extract_numbers(s):
    import re
    
    return [int(x) if x.isdigit() else x for x in re.split('([0-9]+)', s)]

In [31]:
df = pd.read_csv(os.path.join(os.getcwd(), 'a_b_samples_uniform_1.csv'))[['a','b']]
df.head()

,a,b
0,0.890731,1.773921
1,1.306220,1.490305
2,1.653587,0.538929
3,0.666805,1.375998
4,1.105232,1.243956


## we need 3 test statistics
1. $\lambda_{\text{Observed, pred}}$ is between the actually observed (fixed) data and the prediction at $\theta_i$ 

$$ \lambda_{\text{Observed, pred}}(D;\theta_i)=\sum_{\mathcal{O}} \sqrt{ \frac{1}{|b|} \sum_{b \in \mathcal{O}}\left(\frac{\text{MC}_{(b)}\left(\theta_i\right)-\text{Data}_{(b)}}{\Delta_{\text{MC}, b}}\right)^2}$$

where $\text{MC}_{(b)}\left(\theta_i\right)$ denotes the MC prediction for observable $\mathcal{O}$ at bin $b$, which depends on the parameters $\theta_i = \{ a_i, b_i \}$, and $\Delta_{\text{MC} \ b}$ represents the error in the MC prediction in the bin $b$, and $|b|$ represents the number of bins in observable $\mathcal{O}$'s histogram.

2. $\lambda_{\text{shuffled-pred, pred}}$ is between the prediction at $\theta_i$ and the prediction at a shuffled point $\theta^*$

$$ \lambda_{\text{shuffled-pred, pred}}(\mathcal{D};\theta_i) =\sum_{\mathcal{O}} \sqrt{ \frac{1}{|b|} \sum_{b \in \mathcal{O}}\left(\frac{\text{MC}_{(b)}\left(\theta_i\right)-\text{MC}_{(b)}\left(\theta_i^*\right)}{\Delta_{\text{MC}, b}}\right)^2}$$

where $\text{MC}_{(b)}\left(\theta_i^*\right)$ is the MC prediction at a shuffled parameter point $\theta^*$.

3. $\lambda_{\text{simulated-observed, pred}}(\mathcal{D};\theta_i)$

$$ \lambda_{\text{simulated-observed, pred}}(\mathcal{D};\theta_i)=\sum_{\mathcal{O}} \sqrt{ \frac{1}{|b|} \sum_{b \in \mathcal{O}}\left(\frac{\text{MC}_{(b)}\left(\theta_i\right)-\text{SimulatedData}_{(b)}}{\Delta_{\text{MC}, b}}\right)^2}$$